# State tomography

**Classical tomography**  
The simplest classical analogy to estimating a quantum state using tomography is estimating the bias of a coin. The bias, denoted bias by $p$, is analogous to the quantum state in a way that will explained shortly.

If we have access to $n_{\rm Tot}$ independent experiements (or measurements) and observe $n_{\rm H}$ heads then maximum likelihood estimate for the bias of the coin is

$$ p_{\rm Max-Like} = \frac{n_H}{n_{\rm Tot}},$$

and the variance of this estimator is $ {\rm Var}[p_{\rm Max-Like}] = p(1-p)/n_{\rm Tot}$.

The things to learn from this example are:
* it takes many measurements to estiamte $p$, it can't be done in a single shot
* $p_{\rm Max-Like}$ is an estimator, there are other choices of estimators see e.g. [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution#Bayesian_inference) and [Bayes estimator](https://en.wikipedia.org/wiki/Bayes_estimator)


Let's take the analogy a little further. Lets define a "quantum" state 

$$\begin{align}
\rho 
&= \begin{pmatrix} p & 0\\ 0 & 1-p\end{pmatrix} \\
&= \frac{1}{2} (I +z Z)\\
&=\frac{1}{2} \begin{pmatrix} 1+z & 0\\ 0 & 1-z\end{pmatrix},
\end{align}
$$ 
where $I$ and $Z$ are the identity and the Pauli-z matrix. This parameterizes the states along the z-axis of the Bloch sphere


![alt txt](figs/bloch.png)


The relationship between the $Z$ expectation value and the coin bias is

$$z:= \langle Z \rangle = {\rm Tr}[Z \rho] = 2 p -1.$$

In this analogy the pure states $|0\rangle$ and $|1\rangle$ corespond to the coin biases $p=0$ and $p=1$ respectively, all other (mixed) states are convex mixtures of these extremal points.

**Quantum tomography of a single qubit**  

The simplest quantum system to tomograph is a single qubit. Like before we parameterize the state with respect to a set of operators 
$$\begin{align}
\rho 
&= \frac{1}{2} (I+x X+ y Y +z Z)\\
&=\frac{1}{2} \begin{pmatrix} 1+z & x+iy\\ x-iy & 1-z\end{pmatrix},
\end{align}
$$ 
where $x = \langle X \rangle$, $y = \langle Y \rangle$, and $z = \langle Z \rangle$. In the language of our classical coin we have three parameters we need to estimate that are constrained in the following way

$$
0\le x \le 1\\
0\le y \le 1\\ 
0\le z \le 1\\
x^2 + y^2 +z^2 \le 1.
$$

The physics of our system means that our default our measurement gives us the Z basis statistics. We already constructed an estimator to go from the coin flip statistics to the Z expectation: $2p -1$. 


Now we need to, measure the statistics of the operators X and Y. Essentially this means we must rotate our state after we prepare it but before it is measured (or equivalently rotate our measurement basis). If we rotate the state as $\rho\mapsto U\rho U^\dagger$ and then do our usual Z-basis measurement, then this is equivalent to rotating the measured observable as $Z \mapsto U^\dagger Z U$ and keeping our state $\rho$ unchanged. This is the disticntion between the [Heisenberg and Schrödinger pictures](https://en.wikipedia.org/wiki/Heisenberg_picture#Summary_comparison_of_evolution_in_all_pictures). The Heisenberg picture point of view then allows us to see that if we apply a rotation such as $U=R_y(\pi/2)$ then this rotates the observable as $R_y(-\pi/2)ZR_y(+\pi/2)=\cos(\pi/2) Z - \sin(\pi/2) X = -X$. Similarly, we could rotate by $U=R_x(\pi/2)$ to measure the Y observable.  

  


**Quantum state tomography in general**   
Involves measuring all the expectation value of all the operators given by `itertools.product(['X', 'Y', 'Z], repeat=n_qubits)` many times. From these measurements, we can reconstruct a density matrix $\rho$ on `n_qubits`.

**More information**  
See the following references:

[Quantum tomography wikipedia page](https://en.wikipedia.org/wiki/Quantum_tomography)

*Initialization and characterization of open quantum systems*  
Christopher Wood,  
Chapter 3, PhD Thesis, University of Waterloo (2015)  
http://hdl.handle.net/10012/9557  


Introduction to Quantum Gate Set Tomography  
Daniel Greenbaum,  
arXiv:1509.02921 (2015)    
https://arxiv.org/abs/1509.02921  

## Some imports

In [1]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *

## Prepare a state with a `Program`
We'll construct a two-qubit graph state by Hadamarding all qubits and then applying a controlled-Z operation across edges of our graph. In the two-qubit case, there's only one edge. The vector we end up preparing is

$$ |\Psi\rangle = \frac{1}{2}\begin{pmatrix} 1\\ 1 \\ 1\\ -1\end{pmatrix}= {\rm CZ}(0,1){\rm H}(1){\rm H}(0)|0,0\rangle,$$

which corresponds to the state matrix

$$ \rho_{\rm true} = |\Psi\rangle\langle \Psi| = \frac{1}{4}\begin{pmatrix} 1 & 1 & 1 &-1\\ 1 & 1 & 1 &-1 \\ 1 & 1 & 1 &-1\\ -1 & -1 & -1 & 1\end{pmatrix}.$$

In [3]:
# numerical representation of the true state
Psi = (1/2) * np.array([1, 1, 1, -1])
rho_true = np.outer(Psi, Psi.T.conj())

rho_true

array([[ 0.25,  0.25,  0.25, -0.25],
       [ 0.25,  0.25,  0.25, -0.25],
       [ 0.25,  0.25,  0.25, -0.25],
       [-0.25, -0.25, -0.25,  0.25]])

In [4]:
qubits = [0, 1]
state_prep_prog = Program()
for qubit in qubits:
    state_prep_prog += H(qubit)
state_prep_prog += CZ(qubits[0], qubits[1])
print(state_prep_prog)

H 0
H 1
CZ 0 1



## Construct a `ObservablesExperiment` for state tomography
We can print this out to see the 16 measurements we will perform.

In [5]:
from forest.benchmarking.tomography import generate_state_tomography_experiment
experiment = generate_state_tomography_experiment(program=state_prep_prog, qubits=qubits)
print(experiment)

H 0; H 1; CZ 0 1
0: Z0_0 * Z0_1→(1+0j)*X1
1: Z0_0 * Z0_1→(1+0j)*Y1
2: Z0_0 * Z0_1→(1+0j)*Z1
3: Z0_0 * Z0_1→(1+0j)*X0
4: Z0_0 * Z0_1→(1+0j)*X0X1
5: Z0_0 * Z0_1→(1+0j)*X0Y1
6: Z0_0 * Z0_1→(1+0j)*X0Z1
7: Z0_0 * Z0_1→(1+0j)*Y0
8: Z0_0 * Z0_1→(1+0j)*Y0X1
9: Z0_0 * Z0_1→(1+0j)*Y0Y1
10: Z0_0 * Z0_1→(1+0j)*Y0Z1
11: Z0_0 * Z0_1→(1+0j)*Z0
12: Z0_0 * Z0_1→(1+0j)*Z0X1
13: Z0_0 * Z0_1→(1+0j)*Z0Y1
14: Z0_0 * Z0_1→(1+0j)*Z0Z1


### Optional grouping
We can simultaneously estimate some of these observables

In [6]:
from forest.benchmarking.observable_estimation import group_settings
print(group_settings(experiment))

H 0; H 1; CZ 0 1
0: Z0_0 * Z0_1→(1+0j)*X1, Z0_0 * Z0_1→(1+0j)*X0, Z0_0 * Z0_1→(1+0j)*X0X1
1: Z0_0 * Z0_1→(1+0j)*Y1, Z0_0 * Z0_1→(1+0j)*X0Y1
2: Z0_0 * Z0_1→(1+0j)*Z1, Z0_0 * Z0_1→(1+0j)*X0Z1
3: Z0_0 * Z0_1→(1+0j)*Y0, Z0_0 * Z0_1→(1+0j)*Y0X1
4: Z0_0 * Z0_1→(1+0j)*Y0Y1
5: Z0_0 * Z0_1→(1+0j)*Y0Z1
6: Z0_0 * Z0_1→(1+0j)*Z0, Z0_0 * Z0_1→(1+0j)*Z0X1
7: Z0_0 * Z0_1→(1+0j)*Z0Y1
8: Z0_0 * Z0_1→(1+0j)*Z0Z1


## PyQuil will run the tomography programs

In [7]:
from forest.benchmarking.observable_estimation import estimate_observables

qc = get_qc('2q-qvm')
# Over-write full quilc compilation with a much more simple
# version that *only* substitutes gates to Rigetti-native gates.
# We don't want to accidentally compile away our tomography circuit
# or map to different qubits.
from forest.benchmarking.compilation import basic_compile
qc.compiler.quil_to_native_quil = basic_compile

results = list(estimate_observables(qc, experiment))
results

[ExperimentResult[Z0_0 * Z0_1→(1+0j)*X1: -0.088 +- 0.04454786190155483],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Y1: 0.008 +- 0.04471992844359213],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Z1: 0.024 +- 0.04470847794322683],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*X0: 0.024 +- 0.04470847794322683],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*X0X1: -0.032 +- 0.04469845634918503],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*X0Y1: 0.028 +- 0.04470382533967311],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*X0Z1: 1.0 +- 0.0],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Y0: -0.032 +- 0.04469845634918503],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Y0X1: -0.044 +- 0.0446780483011512],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Y0Y1: 1.0 +- 0.0],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Y0Z1: -0.036 +- 0.044692370713579295],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Z0: 0.032 +- 0.04469845634918503],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Z0X1: 1.0 +- 0.0],
 ExperimentResult[Z0_0 * Z0_1→(1+0j)*Z0Y1: -0.016 +- 0.04471563484956912],
 ExperimentResult[Z0_0 *

### We can look at a bunch of numbers...

In [8]:
from forest.benchmarking.tomography import linear_inv_state_estimate
rho = linear_inv_state_estimate(results, qubits=qubits)
np.round(rho, 3)

array([[ 0.245-0.j   ,  0.228+0.002j,  0.256+0.017j, -0.258+0.004j],
       [ 0.228-0.002j,  0.271+0.j   ,  0.242+0.018j, -0.244-0.001j],
       [ 0.256-0.017j,  0.242-0.018j,  0.267-0.j   , -0.272-0.006j],
       [-0.258-0.004j, -0.244+0.001j, -0.272+0.006j,  0.217-0.j   ]])

### Or visualize using Hinton plots

In [9]:
from matplotlib import pyplot as plt
from forest.benchmarking.plotting import hinton
fig, (ax1, ax2) = plt.subplots(1, 2)
hinton(rho_true, ax=ax1)
hinton(rho, ax=ax2)
ax1.set_title('Analytical')
ax2.set_title('Estimated')
fig.tight_layout()

### Matrix norm between true and estimated is low

In [ ]:
np.linalg.norm(rho - rho_true)

## Linear inversion estimate

In [ ]:
from forest.benchmarking.tomography import linear_inv_state_estimate
rho = linear_inv_state_estimate(results, qubits=qubits)

print(np.round(rho, 4))
print('Purity =', np.trace(rho @ rho))
hinton(rho)

## Maximum Liklihood Estimate (MLE) via diluted iterative method

In [ ]:
from forest.benchmarking.tomography import iterative_mle_state_estimate
rho = iterative_mle_state_estimate(results=results, qubits=qubits)
print(np.around(rho, decimals=4))
print('Purity =', np.trace(rho @ rho))
hinton(rho)

## MLE with Max Entropy constraint

In [ ]:
rho = iterative_mle_state_estimate(results=results, qubits=qubits, epsilon=0.5, entropy_penalty=0.005)
print(np.around(rho, decimals=4))
print('Purity =', np.trace(rho @ rho))
hinton(rho)

## MLE with Hedging parameter

In [ ]:
rho = iterative_mle_state_estimate(results=results, qubits=qubits, epsilon=.0001, beta=.61)
print(np.around(rho, decimals=4))
print('Purity = ', np.trace(rho @ rho))
hinton(rho)

## Project an unphysical state to the closest physical state

In [ ]:
from forest.benchmarking.operator_tools.project_state_matrix import project_state_matrix_to_physical
rho_unphys = np.random.uniform(-1, 1, (2, 2)) \
    * np.exp(1.j * np.random.uniform(-np.pi, np.pi, (2, 2)))
rho_phys = project_state_matrix_to_physical(rho_unphys)

fig, (ax1, ax2) = plt.subplots(1, 2)
hinton(rho_unphys, ax=ax1)
hinton(rho_phys, ax=ax2)
ax1.set_title('Unphysical')
ax2.set_title('Physical projection')
fig.tight_layout()

In [ ]:
# Test the wizard method. Example from fig 1 of maximum likelihood minimum effort 
# https://doi.org/10.1103/PhysRevLett.108.070502

eigs = np.diag(np.array(list(reversed([3.0/5, 1.0/2, 7.0/20, 1.0/10, -11.0/20]))))
phys = project_state_matrix_to_physical(eigs)
np.allclose(phys, np.diag([0, 0, 1.0/5, 7.0/20, 9.0/20]))

# Lightweight Bootstrap for functionals of the state

In [ ]:
import forest.benchmarking.distance_measures as dm
from forest.benchmarking.tomography import estimate_variance

In [ ]:
from functools import partial
fast_tomo_est = partial(iterative_mle_state_estimate, epsilon=.0001, beta=.5, tol=1e-3)

**Purity**

In [ ]:
mle_est = estimate_variance(results, qubits, fast_tomo_est, dm.purity,
                            n_resamples=40, project_to_physical=True)
lin_inv_est = estimate_variance(results, qubits, linear_inv_state_estimate, dm.purity,
                                n_resamples=40, project_to_physical=True)
print(mle_est)
print(lin_inv_est)

**Fidelity**

In [ ]:
mle_est = estimate_variance(results, qubits, fast_tomo_est, dm.fidelity,
                            target_state=rho_true, n_resamples=40, project_to_physical=True)
lin_inv_est = estimate_variance(results, qubits, linear_inv_state_estimate, dm.fidelity,
                                target_state=rho_true, n_resamples=40, project_to_physical=True)
print(mle_est)
print(lin_inv_est)